# 버스 승객 승하차 수 전처리 

### 1. 버스 승객수 월별 시간별 승하차 수 데이터      
추정치를 구하기 위해 하루중 1시간단위로 승객의 승하차 비율을 구함.  

In [1]:
# !pip install pymysql
# !pip install sqlalchemy

In [2]:
import pandas as pd
import pymysql
from Load import *
from sqlalchemy import *
from module import *
from bus_preprocessing import *

In [3]:
# 버스 승객수 월별 시간별 데이터
bus_station_time_df= pd.read_csv('월별_시간별_버스승객수_승하차_raw.csv', encoding='utf-8-sig')
total_bus_time_df=bus_month_time_passenger(bus_station_time_df)
# csv로 내보내기 
total_bus_time_df.to_csv("월별_시간별_버스승객수_승하차비율_완성본.csv", encoding='UTF-8-sig')

C:\Users\user\AppData\Local\Temp\ipykernel_31684\2383374866.py:2: DtypeWarning: Columns (3,55) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_station_time_df= pd.read_csv('월별_시간별_버스승객수_승하차_raw.csv', encoding='utf-8-sig')
c:\Users\user\Desktop\CM_리팩토링_git용\전처리\bus_preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ride_df['승차인원 수']=ride_df.sum(axis=1)
c:\Users\user\Desktop\CM_리팩토링_git용\전처리\bus_preprocessing.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

### 2. 버스 승객 승하차 하루 총 승하차 수 데이터 



수집한 일별_버스승객_승하차 데이터를 데이터프레임으로 불러오고 필요한 컬럼들만 추출한다

In [4]:
daily_bus_passenger_df_1 =pd.read_csv("일별_버스승객_승하차_1_raw.csv", encoding='utf-8-sig')
daily_bus_passenger_df_2 =pd.read_csv("일별_버스승객_승하차_2_raw.csv", encoding='utf-8-sig')
daily_bus_passenger_df_3 =pd.read_csv("일별_버스승객_승하차_3_raw.csv", encoding='utf-8-sig')

In [5]:
daily_bus_passenger_list=[daily_bus_passenger_df_1, daily_bus_passenger_df_2, daily_bus_passenger_df_3]
daily_bus_passenger_df=pd.DataFrame()

for daily_bus_passenger_data in daily_bus_passenger_list:
    daily_bus_passenger_data = daily_bus_passenger(daily_bus_passenger_data)
    daily_bus_passenger_df=pd.concat([daily_bus_passenger_df,daily_bus_passenger_data], axis=0)


In [6]:
daily_bus_passenger_df

,RIDE_PASGR_NUM,ALIGHT_PASGR_NUM,월별_노선_정류장_ID
0,12,0,202101_100_110000327
1,36,5,202101_100_110000335
2,19,20,202101_100_107000013
3,17,35,202101_100_107000014
4,55,80,202101_100_107000015
...,...,...,...
7656029,10,76,202305_8762_113000180
7656030,236,4,202305_8762_113000202
7656031,46,69,202305_8762_113000204
7656032,4,86,202305_8762_113000450


### 3. 월별 시간별 승객수 데이터와 일별 총 승객수 데이터를 병합 후 추정치 구하기    
계산식 : 시간별 승객 승하차 비율 X 일별 총 승객 승하차 수 = 매일 시간별 승객 승하차 수          
병합 기준 : 일별 총 승객 승하차 수 (매일매일 시간별로 승객수를 알아야하므로)

In [7]:
# RIGHT MERGE
daily_time_bus_passenger = total_bus_time_df.merge(daily_bus_passenger_df, on='월별_노선_정류장_ID', how='right')

In [8]:
daily_time_bus_passenger=daily_time_bus_passenger.dropna()
daily_time_bus_passenger['USE_MON']=daily_time_bus_passenger['USE_MON'].astype(int)

In [9]:
# 매일 시간별로 승객수 추정치 구하기 
daily_time_bus_passenger_final=calculate_passenger(daily_time_bus_passenger)   

In [10]:
daily_time_bus_passenger_final.to_csv('일별_시간별_버스승객_승하차_추정치.csv',encoding='UTF-8-sig')

# 정류장 주변 인프라 데이터 전처리

### 통계청 API
- https://sgis.kostat.go.kr/developer/html/main.html
- 정류장들이 속한 지역의 특징을 확인하기 위해 읍면동 단위로 데이터를 수집함
- 인구수, 가구수, 사업체 수, 사업체 종사자 수

In [4]:
import sys
sys.path.append('./Data')
sys.path.append('./Data/csv')

from Extract import get_sgis_accessToken, get_population_data
from preprocessing import *

In [5]:
sgis_df = get_population_data()
sgis_df.info()
sgis_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   adm_cd                  426 non-null    object
 1   adm_nm                  426 non-null    object
 2   tot_family              426 non-null    object
 3   tot_ppltn               426 non-null    object
 4   corp_cnt                426 non-null    object
 5   employee_cnt            426 non-null    object
 6   population_code_15to64  426 non-null    object
 7   household_cnt_family    426 non-null    object
 8   household_cnt_alone     426 non-null    object
dtypes: object(9)
memory usage: 30.1+ KB


,adm_cd,adm_nm,tot_family,tot_ppltn,corp_cnt,employee_cnt,population_code_15to64,household_cnt_family,household_cnt_alone
0,11010530,사직동,3839,8263,5198,53698,5899,2217,1487
1,11010540,삼청동,1031,2271,990,4840,1549,618,380
2,11010550,부암동,3773,9153,876,3764,6563,2518,1129
3,11010560,평창동,6265,16832,1290,4456,11838,4846,1302
4,11010570,무악동,2883,7935,358,1669,5474,2357,494
...,...,...,...,...,...,...,...,...,...
421,11250730,천호2동,15701,33386,4639,14258,24611,8837,6378
422,11250740,길동,19509,43844,5293,24415,32738,12109,6909
423,11250750,강일동,8887,24113,979,4144,16714,6988,1800
424,11250760,상일1동,10984,31528,1309,5932,22206,9435,1441


### KAKAO 지도 API
- 각 정류장 반경 500m 내에 있는 인프라 데이터를 수집함
- 학원, 은행, 유치원, 마트, 식당, 학교, 대학교, 지하철, 관광명소, 카페, 병원, 문화시설, 대학병원, 공공시설 (단위 : 개수)

In [6]:
from Extract import kakao_infra
import pandas as pd

kakao_df = pd.read_csv('./Data/csv/df_kakao_infra.csv', encoding = 'utf-8')

In [19]:
kakao_df.info()
kakao_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12570 entries, 0 to 12569
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NODE_ID            12570 non-null  int64  
 1   ARS_ID             12570 non-null  int64  
 2   정류소명               12570 non-null  object 
 3   X좌표                12570 non-null  float64
 4   Y좌표                12570 non-null  float64
 5   법정동코드              12570 non-null  int64  
 6   법정동_구              12570 non-null  object 
 7   법정동                12570 non-null  object 
 8   행정동코드              12570 non-null  int64  
 9   행정동                12570 non-null  object 
 10  academy_cnt        12570 non-null  int64  
 11  kindergarten_cnt   12570 non-null  int64  
 12  mart_cnt           12570 non-null  int64  
 13  restaurant_cnt     12570 non-null  int64  
 14  school_cnt         12570 non-null  int64  
 15  subway_cnt         12570 non-null  int64  
 16  tour_cnt           125

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,행정동코드,행정동,...,kindergarten_cnt,mart_cnt,restaurant_cnt,school_cnt,subway_cnt,tour_cnt,cafe_cnt,hospital_cnt,culture_cnt,public_office_cnt
0,100000001,1001,종로2가사거리,126.987750,37.569765,1111013800,종로구,종로2가,1111061500,종로1.2.3.4가동,...,8,0,1486,0,5,16,403,62,68,10
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,1111016600,종로구,연건동,1111061500,종로1.2.3.4가동,...,4,0,195,3,0,26,88,12,19,1
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,1111017200,종로구,명륜4가,1111065000,혜화동,...,2,1,566,3,1,19,214,32,146,2
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,1111013500,종로구,관철동,1111061500,종로1.2.3.4가동,...,10,0,1404,1,3,13,351,73,52,7
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,1111016900,종로구,혜화동,1111065000,혜화동,...,8,3,594,4,2,4,235,36,145,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12565,124900137,25682,9호선중앙보훈병원역1번출구,127.147539,37.527911,1174010600,강동구,둔촌동,1174070000,둔촌2동,...,8,0,50,3,1,1,11,17,0,1
12566,129000058,16349,김포여객터미널,126.786594,37.599064,4157025325,김포시,고촌읍,4157025300,고촌읍,...,0,0,46,0,0,1,11,0,1,2
12567,129000059,16359,김포화물터미널,126.793410,37.591997,4157025325,김포시,고촌읍,4157025300,고촌읍,...,0,0,7,0,0,0,3,0,0,0
12568,129000060,16358,김포화물터미널,126.794492,37.591772,4157025325,김포시,고촌읍,4157025300,고촌읍,...,0,0,6,0,0,0,2,0,0,0


### 인프라 데이터 전처리
- 법정동, 행정동 매칭
- 법정동 코드 기준으로 `sgis_df`, `kakao_df` 병합

In [7]:
# 통계청 df, 카카오 df 병합 과정 코드 추가

# 통계청 데이터 법정동 코드 기준으로 변환
sgis_df = get_population_data_by_bubjeong()

# 통계청 데이터, 카카오 지도 api 데이터 법정동 기준으로 병합
sgis_kakao_merge_df = preprocessing_infra()

# sgis_kakao_merge_df 에 승차, 하차 수 데이터 병합
infra_final_df = get_final_infra_df()

c:\Users\eunbi\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\eunbi\Desktop\파이널 리팩토링\Resurrection_Of_Squirrel_Bus\src\./Data\preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_mod['adm_code'] = [code +'0' if len(code) < 8 else code for code in code_mod['행정구역코드']]
c:\Users\eunbi\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\eunbi\Desktop\파이널 리팩토링\Resurrection_Of_Squirrel_Bus\src\./Data\preprocessing.py:22: Set

In [9]:
infra_final_df.info()
infra_final_df

<class 'pandas.core.frame.DataFrame'>
Index: 12548 entries, 0 to 12565
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NODE_ID            12548 non-null  object 
 1   정류소명               12548 non-null  object 
 2   X좌표                12548 non-null  object 
 3   Y좌표                12548 non-null  object 
 4   법정동코드              12548 non-null  object 
 5   법정동_구              12548 non-null  object 
 6   법정동                12548 non-null  object 
 7   academy_cnt        12548 non-null  int32  
 8   kindergarten_cnt   12548 non-null  int32  
 9   mart_cnt           12548 non-null  int32  
 10  restaurant_cnt     12548 non-null  int32  
 11  school_cnt         12548 non-null  int32  
 12  university_cnt     12548 non-null  int32  
 13  subway_cnt         12548 non-null  int32  
 14  tour_cnt           12548 non-null  int32  
 15  cafe_cnt           12548 non-null  int32  
 16  hospital_cnt       12548 no

,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,academy_cnt,kindergarten_cnt,mart_cnt,...,hospital_cnt,culture_cnt,univ_hospital_cnt,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10
0,100000001,종로2가사거리,126.9877499,37.56976513,1111013800,종로구,종로2가,184,8,0,...,62,68,0,10,4091,0.399899,6.383245,148,84743,52050
1,100000002,창경궁.서울대학교병원,126.996566,37.57918302,1111016600,종로구,연건동,2,4,0,...,12,19,0,1,5952,0.280961,10.546958,1943,387688,712320
2,100000003,명륜3가.성대입구,126.9983401,37.58267117,1111017200,종로구,명륜4가,48,2,1,...,32,146,0,2,1495,0.278595,4.017017,2336,406570,572853
3,100000004,종로2가.삼일교,126.9876131,37.56857927,1111013500,종로구,관철동,165,10,0,...,73,52,0,7,4091,0.399899,6.383245,148,22643,120447
4,100000005,혜화동로터리.여운형활동터,127.001744,37.586243,1111016900,종로구,혜화동,71,8,3,...,36,145,0,4,1495,0.278595,4.017017,2336,294069,402594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12561,124900133,삼익가든아파트,127.14854,37.54630481,1174010100,강동구,명일동,83,19,0,...,23,0,0,1,12523,0.059752,3.497068,29747,5100,269
12562,124900134,삼익가든아파트,127.1488322,37.54628161,1174010100,강동구,명일동,89,21,0,...,21,0,0,1,12523,0.059752,3.497068,29747,307,2914
12563,124900135,둔촌신성미소지움아파트,127.144479,37.52674338,1174010600,강동구,둔촌동,23,7,0,...,14,0,0,1,10704,0.093701,5.314796,18569,5431,689
12564,124900136,둔촌신성미소지움아파트,127.1444012,37.52658385,1174010600,강동구,둔촌동,23,7,0,...,14,0,0,1,10704,0.093701,5.314796,18569,640,2263


### 전처리 완료된 데이터프레임 불러오기

In [16]:
infra_df = pd.read_csv('./Data/csv/tb_infra_population.csv', encoding = 'utf-8')

In [17]:
infra_df

,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동_x,academy_cnt,bank_cnt,kindergarten_cnt,...,public_office_cnt,RIDE_SUM_6_10,ALIGHT_SUM_6_10,tot_family,tot_ppltn,corp_cnt,employee_cnt,population_code_15to64,household_cnt_family,household_cnt_alone
0,100000001,종로2가사거리,126.987750,37.569765,1111013800,종로구,종로2가,185,105,8,...,10,84743,52050,125,204,641,4091,149,37,82
1,100000388,종로2가,126.985543,37.570139,1111013800,종로구,종로2가,170,122,11,...,13,153539,430217,125,204,641,4091,149,37,82
2,100000389,종로2가,126.986248,37.570257,1111013800,종로구,종로2가,179,121,10,...,12,397559,675551,125,204,641,4091,149,37,82
3,100000087,종로3가,126.988937,37.570011,1111013800,종로구,종로2가,188,79,4,...,8,61645,59815,125,204,641,4091,149,37,82
4,100000090,종로2가,126.986490,37.569962,1111013800,종로구,종로2가,178,124,10,...,11,61645,59815,125,204,641,4091,149,37,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12543,124900078,선사초등학교,127.128852,37.554388,1174010700,강동구,암사동,72,10,20,...,4,7973,1504,26215,65610,4337,12152,47206,19061,6739
12544,124900079,암사역,127.127673,37.550931,1174010700,강동구,암사동,92,20,27,...,4,17600,15885,26215,65610,4337,12152,47206,19061,6739
12545,124000268,천호우성아파트,127.137766,37.550195,1174010700,강동구,암사동,41,8,19,...,3,61645,59815,26215,65610,4337,12152,47206,19061,6739
12546,124900108,암사삼성.현대아파트,127.128549,37.553318,1174010700,강동구,암사동,85,15,25,...,4,16411,7956,26215,65610,4337,12152,47206,19061,6739


### 일기예보 API
- 공공데이터 포털에서 제공하는 일기예보 api를 활용해 기후 데이터를 추출함
- 기온, 풍속, 풍향, 강수량, 습도

In [20]:
from Extract import kakao_infra
import pandas as pd

weather_df = pd.read_csv('./Data/csv/0728_weather.csv', encoding = 'utf-8')

In [21]:
weather_df.info()
weather_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   raw_x   148 non-null    float64
 1   raw_y   148 non-null    float64
 2   X       148 non-null    int64  
 3   Y       148 non-null    int64  
 4   time    148 non-null    int64  
 5   TMP     148 non-null    int64  
 6   VEC     148 non-null    int64  
 7   WSD     148 non-null    float64
 8   PCP     148 non-null    int64  
 9   REH     148 non-null    int64  
 10  Date    148 non-null    object 
dtypes: float64(3), int64(7), object(1)
memory usage: 12.8+ KB


,raw_x,raw_y,X,Y,time,TMP,VEC,WSD,PCP,REH,Date
0,126.822235,37.547383,57,126,600,26,7,0.8,0,90,2023-07-29
1,126.822235,37.547383,57,126,700,28,27,0.7,0,85,2023-07-29
2,126.822235,37.547383,57,126,800,29,354,1.0,0,75,2023-07-29
3,126.822235,37.547383,57,126,900,31,356,1.3,0,65,2023-07-29
4,126.824277,37.547851,57,126,600,26,7,0.8,0,90,2023-07-29
...,...,...,...,...,...,...,...,...,...,...,...
143,126.886518,37.580333,58,127,900,31,355,1.2,0,65,2023-07-29
144,126.884046,37.581780,58,127,600,27,16,0.7,0,85,2023-07-29
145,126.884046,37.581780,58,127,700,28,40,0.8,0,80,2023-07-29
146,126.884046,37.581780,58,127,800,29,360,0.9,0,75,2023-07-29
